# FrontPipe Debug Notebook
Use this to test the system locally before wiring real Outlook/FTP/API.

**What this does**
- Loads the sample YAML
- Runs a single orchestrator pass (moves files from `dev_fixtures/email_samples` to `drop/Data Files/<date>`)
- Regenerates the status page

> Tip: Place test files in `dev_fixtures/email_samples/` and tweak regexes in `config/frontpipe.sample.yaml`.

In [ ]:
from datetime import datetime, timezone
import os, json
base = os.path.abspath(".")
print("Base:", base)

In [ ]:
from frontpipe.orchestrator import run_once
res = run_once("config/frontpipe.sample.yaml", ".", now_utc=datetime(2025,8,14,tzinfo=timezone.utc))
res["target_date"], len(res["master_rows"])

In [ ]:
# Show master rows
for r in res["master_rows"]:
    print(r["counterparty"], r["stream"], r["status"], r.get("note",""), r.get("saved_path",""))

In [ ]:
# Regenerate status page from saved state
from frontpipe.state_store import MasterState
from frontpipe.reporter import write_status
state = MasterState("runtime/state")
rows = state.load(res["target_date"])
write_status("runtime/status.html", rows)
print("Wrote runtime/status.html")

### Arrival window override for testing
In the YAML, set `test_ignore_arrival_window: true` per feed while testing.
Production code should enforce windows inside collectors—kept permissive here to simplify local tests.

### Debug playbook
1. Edit `config/frontpipe.sample.yaml` patterns to match your test file names.
2. Drop dummy files into `dev_fixtures/email_samples/`.
3. Rerun the orchestrator cell.
4. Open `runtime/status.html` to confirm statuses.
5. If a feed requires two attachments, ensure both files exist; status remains `pending` until both are found.